# Worksheet 04

Name:  
UID: 

### Topics

- Distance & Similarity
- Cost Functions
- K means

### Distance & Similarity

#### Part 1

a) In the minkowski distance, describe what the parameters p and d are.

d is the number of dimensions.
p is the magnitude of power mean.

b) In your own words describe the difference between the Euclidean distance and the Manhattan distance.

Euclidean distance is the shortest distance between two points in an n-dimensional space.
Manhattan distance is the distance between two points - traveling one unit in one dimension at a time - for a n-dimensional space.

Consider A = (0, 0) and B = (1, 1). When:

- p = 1, d(A, B) = 2
- p = 2, d(A, B) = $\sqrt{2} = 1.41$
- p = 3, d(A, B) = $2^{1/3} = 1.26$
- p = 4, d(A, B) = $2^{1/4} = 1.19$

c) Describe what you think distance would look like when p is very large.

The distance would be close to 1.

d) Is the minkowski distance still a distance function when p < 1? Expain why / why not.

No it isn't. Because it violates one of the conditions of a distance function: D(A,C)< A(A,B)+D(B,C)

e) when would you use cosine similarity over the euclidan distance?

When direction matters more than magnitude.

f) what does the jaccard distance account for that the manhattan distance doesn't?

Jaccard distance accounts for the size of intersection over union of both the points.

#### Part 2

Consider the following two sentences:

In [3]:
s1 = "hello my name is Alice"  
s2 = "hello my name is Bob"

using the union of words from both sentences, we can represent each sentence as a vector. Each element of the vector represents the presence or absence of the word at that index.

In this example, the union of words is ("hello", "my", "name", "is", "Alice", "Bob") so we can represent the above sentences as such:

In [4]:
v1 = [1,    1, 1,   1, 1,    0]
#     hello my name is Alice
v2 = [1,    1, 1,   1, 0, 1]
#     hello my name is    Bob

Programmatically, we can do the following:

In [5]:
corpus = [s1, s2]
all_words = list(set([item for x in corpus for item in x.split()]))
print(all_words)
v1 = [1 if x in s1 else 0 for x in all_words]
print(v1)

['Alice', 'name', 'Bob', 'hello', 'is', 'my']
[1, 1, 0, 1, 1, 1]


Let's add a new sentence to our corpus:

In [6]:
s3 = "hi my name is Claude"
corpus.append(s3)

a) What is the new union of words used to represent s1, s2, and s3?

In [7]:
all_words = list(set([item for x in corpus for item in x.split()]))
print(all_words)


['hi', 'Alice', 'name', 'Bob', 'Claude', 'hello', 'is', 'my']


b) Represent s1, s2, and s3 as vectors as above, using this new set of words.

In [9]:
v1 = [1 if x in s1 else 0 for x in all_words]
v2 = [1 if x in s2 else 0 for x in all_words]
v3 = [1 if x in s3 else 0 for x in all_words]
print(s1, v1)
print(s2, v2)
print(s3, v3)

hello my name is Alice [0, 1, 1, 0, 0, 1, 1, 1]
hello my name is Bob [0, 0, 1, 1, 0, 1, 1, 1]
hi my name is Claude [1, 0, 1, 0, 1, 0, 1, 1]


c) Write a function that computes the manhattan distance between two vectors. Which pair of vectors are the most similar under that distance function?

In [12]:
# given the dimensions of v1 and v2 are the same.
def manhattan_dist(v1, v2):
    dist = 0
    for i in range(len(v1)):
        dist += abs(v1[i]-v2[i])
    return dist
print(manhattan_dist(v1, v2))
print(manhattan_dist(v2, v3))
print(manhattan_dist(v3, v1))

2
4
4


V1 and V2 are the most similar as they have the least distance between them = 2.

d) Create a matrix of all these vectors (row major) and add the following sentences in vector form:

- "hi Alice"
- "hello Claude"
- "Bob my name is Claude"
- "hi Claude my name is Alice"
- "hello Bob"

In [14]:
s1 = "hi Alice"
s2 =  "hello Claude"
s3 =  "Bob my name is Claude"
s4 =  "hi Claude my name is Alice"
s5 =  "hello Bob"
corpus = [s1, s2, s3, s4, s5]
all_words = list(set([item for x in corpus for item in x.split()]))
matrix = [[1 if x in s else 0 for x in all_words] for s in corpus]
print(matrix)

[[1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0, 1, 1], [1, 1, 1, 0, 1, 0, 1, 1], [0, 0, 0, 1, 0, 1, 0, 0]]


e) How many rows and columns does this matrix have?

In [20]:
print("rows =", len(matrix), ", cols =", len(matrix[0]))

rows = 5 , cols = 8


f) When using the Manhattan distance, which two sentences are the most similar?

In [25]:
# take a large value as we need to adjust this to min
mostSimilar = 1000000
leastOne, leastTwo = -1, -1
for i in range(len(matrix)):
    for j in range(i, len(matrix)):
        if i!= j:
            v1 = matrix[i]
            v2 = matrix[j]
            if(mostSimilar>manhattan_dist(v1, v2)):
                mostSimilar = manhattan_dist(v1, v2)
                leastOne = i
                leastTwo = j
print("The most similar ones are:")
print("->",corpus[leastOne], '\n->',corpus[leastTwo])

The most similar ones are:
-> hello Claude 
-> hello Bob


### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

Iteration 1:

Cluster 1 -> 0, .5

Cluster 2 -> 1.5, 2, 6, 6.5, 7 

Iteration 2:

New centroids = [0.25, 11.5]

Cluster 1 -> 0, .5

Cluster 2 -> 1.5, 2, 6, 6.5, 7 

They will never converge

b) Describe in plain english what the cost function for k means is.

Way to evaluateCost Function is is the measure of how different far the points are from the centroid. More the cost function, lesser optimal it is.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Different intial values of centroids can produce different solutions.

d) Does Lloyd's Algorithm always converge? Why / why not?

Lloyd's algorithm doesn't always converge. There can be a case where the centroids remain in the same place but the clusters change between themselves in each iteration.
When this happens, it results in an infinite loop and the algorithm never converges.